In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, rdFingerprintGenerator, rdMolDescriptors
from padelpy import padeldescriptor

In [15]:
df_train = pd.read_csv("/home/users/akshay/PCPpred/data/Train.csv")
df_train

,ID,SMILES,Permeability,Sequence,MolWt
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,"['F', 'meL', 'A', 'meL', 'Ser(tBu)', 'meL', 'm...",1773.325
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,"['F', 'meF', 'P', 'L', 'Me_Phe(3-Cl)', 'T', 'L...",1745.057
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,"['Sar', 'F', 'meL', 'meL', 'F', 'meL', 'L', 'm...",1733.267
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,"['Me_Bal', 'F', 'meF', 'meL', 'T', 'meA', 'meL...",1725.281
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,"['meA', 'meL', 'meF', 'T', 'meL', 'meL', 'I', ...",1723.309
...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,"['A', 'L', 'Mono76']",402.539
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,"['A', 'L', 'H2NEt_Phe']",374.485
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,"['dL', 'S', 'Mono85']",370.494
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,"['dS', 'V', 'Mono85']",356.467


In [16]:
df_train_cmfp = df_train[['ID','SMILES','Permeability']]
df_train_cmfp

,ID,SMILES,Permeability
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0
...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3


In [17]:
def generate_count_morgan_fp(smiles, radius=2, nBits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        # Count Morgan fingerprint generator
        mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
        count_fp = mfpgen.GetCountFingerprint(mol)
        
        # Convert the count fingerprint to a dense array
        dense_fp = np.zeros((nBits,), dtype=int)
        for bit, count in count_fp.GetNonzeroElements().items():
            dense_fp[bit] = count
            
        return dense_fp
    else:
        return None

In [18]:
df_train_cmfp.loc[:,'count_morgan_fp'] = df_train_cmfp['SMILES'].apply(generate_count_morgan_fp)
df_train_cmfp['count_morgan_fp']

/tmp/ipykernel_4093517/1551993060.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_cmfp.loc[:,'count_morgan_fp'] = df_train_cmfp['SMILES'].apply(generate_count_morgan_fp)


0       [0, 9, 1, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 11, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
2       [0, 7, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 10, 1, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
4       [0, 14, 1, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
                              ...                        
5563    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5564    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5565    [0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5566    [0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5567    [0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: count_morgan_fp, Length: 5568, dtype: object

In [19]:
df = pd.DataFrame(df_train_cmfp['count_morgan_fp'].tolist(), index=df_train_cmfp.index)

df_train_cmfp = pd.concat([df_train_cmfp, df.add_prefix('count_fp_')], axis=1)
df_train_cmfp.shape

(5568, 2052)

In [20]:
df_train_cmfp.drop('count_morgan_fp', axis=1, inplace=True)
df_train_cmfp.shape

(5568, 2051)

In [21]:
df_train_cmfp.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/count_morgan_fp_train.csv', index=False)

In [22]:
df_test = pd.read_csv("/home/users/akshay/PCPpred/data/Test.csv")
df_test_cmfp = df_test[['ID','SMILES','Permeability']]
df_test_cmfp.loc[:,'count_morgan_fp'] = df_test_cmfp['SMILES'].apply(generate_count_morgan_fp)
df_test_cmfp['count_morgan_fp']
df = pd.DataFrame(df_test_cmfp['count_morgan_fp'].tolist(), index=df_test_cmfp.index)

df_test_cmfp = pd.concat([df_test_cmfp, df.add_prefix('count_fp_')], axis=1)
df_test_cmfp.drop('count_morgan_fp', axis=1, inplace=True)
df_test_cmfp.shape

/tmp/ipykernel_4093517/3889528555.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_cmfp.loc[:,'count_morgan_fp'] = df_test_cmfp['SMILES'].apply(generate_count_morgan_fp)


(1392, 2051)

In [23]:
df_test_cmfp.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/count_morgan_fp_test.csv', index=False)

In [24]:
def generate_fingerprints(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        # Create a Morgan fingerprint generator
        mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
        
        # Generate the Morgan fingerprint as a bit vector
        morgan_fp = mfpgen.GetFingerprint(mol)
        
        # Convert the bit vector to a dense array
        dense_fp = np.zeros((2048,), dtype=int)
        for bit in range(2048):
            dense_fp[bit] = morgan_fp[bit]
        
        return dense_fp
    else:
        print("Invalid SMILES string.")
        return None

In [25]:
df_train = pd.read_csv("/home/users/akshay/PCPpred/data/Train.csv")
df_mfp_train = df_train[['ID','SMILES','Permeability']]
df_mfp_train.loc[:,'Morganfingerprints'] = df_mfp_train['SMILES'].apply(generate_fingerprints)
df = pd.DataFrame(df_mfp_train['Morganfingerprints'].tolist(), index=df_mfp_train.index)

df_mfp_train = pd.concat([df_mfp_train, df.add_prefix('Morgan_fp_')], axis=1)
df_mfp_train.drop('Morganfingerprints', axis=1, inplace=True)
df_mfp_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/morgan_fp_train.csv', index=False)
print(df_mfp_train.shape)
df_mfp_train

/tmp/ipykernel_4093517/2720930349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mfp_train.loc[:,'Morganfingerprints'] = df_mfp_train['SMILES'].apply(generate_fingerprints)


(5568, 2051)


,ID,SMILES,Permeability,Morgan_fp_0,Morgan_fp_1,Morgan_fp_2,Morgan_fp_3,Morgan_fp_4,Morgan_fp_5,Morgan_fp_6,...,Morgan_fp_2038,Morgan_fp_2039,Morgan_fp_2040,Morgan_fp_2041,Morgan_fp_2042,Morgan_fp_2043,Morgan_fp_2044,Morgan_fp_2045,Morgan_fp_2046,Morgan_fp_2047
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df_test = pd.read_csv("/home/users/akshay/PCPpred/data/Test.csv")
df_mfp_test = df_test[['ID','SMILES','Permeability']]
df_mfp_test.loc[:,'Morganfingerprints'] = df_mfp_test['SMILES'].apply(generate_fingerprints)
df = pd.DataFrame(df_mfp_test['Morganfingerprints'].tolist(), index=df_mfp_test.index)

df_mfp_test = pd.concat([df_mfp_test, df.add_prefix('Morgan_fp_')], axis=1)
df_mfp_test.drop('Morganfingerprints', axis=1, inplace=True)
df_mfp_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/morgan_fp_test.csv', index=False)
print(df_mfp_test.shape)
df_mfp_test

/tmp/ipykernel_4093517/185665962.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mfp_test.loc[:,'Morganfingerprints'] = df_mfp_test['SMILES'].apply(generate_fingerprints)


(1392, 2051)


,ID,SMILES,Permeability,Morgan_fp_0,Morgan_fp_1,Morgan_fp_2,Morgan_fp_3,Morgan_fp_4,Morgan_fp_5,Morgan_fp_6,...,Morgan_fp_2038,Morgan_fp_2039,Morgan_fp_2040,Morgan_fp_2041,Morgan_fp_2042,Morgan_fp_2043,Morgan_fp_2044,Morgan_fp_2045,Morgan_fp_2046,Morgan_fp_2047
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
#Padel fingerprints
import glob
import os
directory = '/home/users/akshay/PCPpred/fingerprints_xml'
pattern = '*.xml'  

xml_files = glob.glob(os.path.join(directory, pattern))
xml_files.sort()
xml_files

['/home/users/akshay/PCPpred/fingerprints_xml/AtomPairs2DFingerprintCount.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/AtomPairs2DFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/EStateFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/ExtendedFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/Fingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/GraphOnlyFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/KlekotaRothFingerprintCount.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/KlekotaRothFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/MACCSFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/PubchemFingerprinter.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/SubstructureFingerprintCount.xml',
 '/home/users/akshay/PCPpred/fingerprints_xml/SubstructureFingerprinter.xml']

In [28]:
FP_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'Extended',
 'Fingerprinter',
 'Graphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

In [29]:
fp = dict(zip(FP_list, xml_files))
fp

{'AtomPairs2DCount': '/home/users/akshay/PCPpred/fingerprints_xml/AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': '/home/users/akshay/PCPpred/fingerprints_xml/AtomPairs2DFingerprinter.xml',
 'EState': '/home/users/akshay/PCPpred/fingerprints_xml/EStateFingerprinter.xml',
 'Extended': '/home/users/akshay/PCPpred/fingerprints_xml/ExtendedFingerprinter.xml',
 'Fingerprinter': '/home/users/akshay/PCPpred/fingerprints_xml/Fingerprinter.xml',
 'Graphonly': '/home/users/akshay/PCPpred/fingerprints_xml/GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': '/home/users/akshay/PCPpred/fingerprints_xml/KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': '/home/users/akshay/PCPpred/fingerprints_xml/KlekotaRothFingerprinter.xml',
 'MACCS': '/home/users/akshay/PCPpred/fingerprints_xml/MACCSFingerprinter.xml',
 'PubChem': '/home/users/akshay/PCPpred/fingerprints_xml/PubchemFingerprinter.xml',
 'SubstructureCount': '/home/users/akshay/PCPpred/fingerprints_xml/SubstructureFingerprintCount.xml',
 'Substruct

In [30]:
df_train = pd.read_csv("/home/users/akshay/PCPpred/data/Train.csv")
df_smi = df_train[['SMILES','ID']]
df_smi.to_csv('/home/users/akshay/PCPpred/data/train.smi', sep='\t', index=False, header=False)
df_smi

,SMILES,ID
0,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,915
1,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,888
2,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,593
3,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,916
4,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,900
...,...,...
5563,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,2469
5564,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,2467
5565,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,2512
5566,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,2511


In [33]:
#Substructure
fingerprint = 'Substructure'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
descriptors

,Name,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,SubFP8,SubFP9,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,915,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,888,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,593,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,916,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,900,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
5564,2467,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
5565,2512,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
5566,2511,1,1,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1


In [34]:
df_Substructure_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_Substructure_train.head()

,ID,SMILES,Permeability,Name,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,915,1,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,888,1,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,593,1,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,916,1,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,900,1,1,1,0,0,0,...,0,0,1,1,1,0,0,0,0,1


In [35]:
df_Substructure_train.drop(['Name'],axis=1,inplace=True)
df_Substructure_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/Substructure_train.csv', index=False)
df_Substructure_train

,ID,SMILES,Permeability,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1


In [36]:
df_test = pd.read_csv("/home/users/akshay/PCPpred/data/Test.csv")
df_smi = df_test[['SMILES','ID']]
df_smi.to_csv('/home/users/akshay/PCPpred/data/test.smi', sep='\t', index=False, header=False)
fingerprint = 'Substructure'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_Substructure_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
# df_Substructure_train.head()
df_Substructure_test.drop(['Name'],axis=1,inplace=True)
df_Substructure_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/Substructure_test.csv', index=False)
df_Substructure_test

,ID,SMILES,Permeability,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,1,1,1,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,1,1,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1


In [37]:
#CountSubstructure
fingerprint = 'SubstructureCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_SubstructureCount_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_SubstructureCount_train.head()

,ID,SMILES,Permeability,Name,SubFPC1,SubFPC2,SubFPC3,SubFPC4,SubFPC5,SubFPC6,...,SubFPC298,SubFPC299,SubFPC300,SubFPC301,SubFPC302,SubFPC303,SubFPC304,SubFPC305,SubFPC306,SubFPC307
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,915,18.0,13.0,6.0,0.0,0.0,0.0,...,0.0,0.0,90.0,90.0,30.0,0.0,0.0,0.0,0.0,70.0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,888,12.0,15.0,5.0,0.0,0.0,0.0,...,0.0,0.0,82.0,82.0,36.0,0.0,0.0,0.0,0.0,71.0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,593,11.0,15.0,5.0,0.0,0.0,0.0,...,0.0,0.0,84.0,84.0,32.0,0.0,0.0,0.0,0.0,72.0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,916,19.0,12.0,7.0,0.0,0.0,0.0,...,0.0,0.0,90.0,90.0,28.0,0.0,0.0,0.0,0.0,64.0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,900,19.0,13.0,8.0,0.0,0.0,0.0,...,0.0,0.0,90.0,90.0,37.0,0.0,0.0,0.0,0.0,65.0


In [38]:
df_SubstructureCount_train.drop(['Name'],axis=1,inplace=True)
df_SubstructureCount_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/SubstructureCount_train.csv', index=False)
df_SubstructureCount_train

,ID,SMILES,Permeability,SubFPC1,SubFPC2,SubFPC3,SubFPC4,SubFPC5,SubFPC6,SubFPC7,...,SubFPC298,SubFPC299,SubFPC300,SubFPC301,SubFPC302,SubFPC303,SubFPC304,SubFPC305,SubFPC306,SubFPC307
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,18.0,13.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,90.0,90.0,30.0,0.0,0.0,0.0,0.0,70.0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,12.0,15.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,82.0,82.0,36.0,0.0,0.0,0.0,0.0,71.0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,11.0,15.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,84.0,84.0,32.0,0.0,0.0,0.0,0.0,72.0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,19.0,12.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,90.0,90.0,28.0,0.0,0.0,0.0,0.0,64.0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,19.0,13.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,90.0,90.0,37.0,0.0,0.0,0.0,0.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,3.0,4.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,21.0,21.0,4.0,0.0,0.0,0.0,0.0,17.0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,3.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,19.0,19.0,4.0,0.0,0.0,0.0,0.0,17.0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,3.0,5.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,23.0,23.0,3.0,0.0,0.0,0.0,0.0,11.0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,3.0,4.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,22.0,22.0,2.0,0.0,0.0,0.0,0.0,11.0


In [39]:
fingerprint = 'SubstructureCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_SubstructureCount_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
df_SubstructureCount_test.drop(['Name'],axis=1,inplace=True)
df_SubstructureCount_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/SubstructureCount_test.csv', index=False)
df_SubstructureCount_test

,ID,SMILES,Permeability,SubFPC1,SubFPC2,SubFPC3,SubFPC4,SubFPC5,SubFPC6,SubFPC7,...,SubFPC298,SubFPC299,SubFPC300,SubFPC301,SubFPC302,SubFPC303,SubFPC304,SubFPC305,SubFPC306,SubFPC307
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,17.0,13.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,88.0,88.0,34.0,0.0,0.0,0.0,0.0,70.0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,19.0,12.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,89.0,89.0,25.0,0.0,0.0,0.0,0.0,64.0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,17.0,11.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,86.0,86.0,35.0,0.0,0.0,0.0,0.0,70.0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,11.0,14.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,80.0,80.0,28.0,0.0,0.0,0.0,0.0,73.0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,16.0,12.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,86.0,86.0,20.0,0.0,0.0,0.0,0.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,3.0,6.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,23.0,23.0,4.0,0.0,0.0,0.0,0.0,17.0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,3.0,6.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,23.0,23.0,4.0,0.0,0.0,0.0,0.0,17.0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,3.0,4.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,21.0,21.0,4.0,0.0,0.0,0.0,0.0,17.0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,1.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,23.0,23.0,2.0,0.0,0.0,0.0,0.0,16.0


In [40]:
#MACCS
fingerprint = 'MACCS'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_MACCS_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_MACCS_train.head()
df_MACCS_train.drop(['Name'],axis=1,inplace=True)
df_MACCS_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/MACCS_train.csv', index=False)
df_MACCS_train

,ID,SMILES,Permeability,MACCSFP1,MACCSFP2,MACCSFP3,MACCSFP4,MACCSFP5,MACCSFP6,MACCSFP7,...,MACCSFP157,MACCSFP158,MACCSFP159,MACCSFP160,MACCSFP161,MACCSFP162,MACCSFP163,MACCSFP164,MACCSFP165,MACCSFP166
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,1,1,0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,1,1,0


In [41]:
fingerprint = 'MACCS'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_MACCS_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
df_MACCS_test.drop(['Name'],axis=1,inplace=True)
df_MACCS_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/MACCS_test.csv', index=False)
df_MACCS_test

,ID,SMILES,Permeability,MACCSFP1,MACCSFP2,MACCSFP3,MACCSFP4,MACCSFP5,MACCSFP6,MACCSFP7,...,MACCSFP157,MACCSFP158,MACCSFP159,MACCSFP160,MACCSFP161,MACCSFP162,MACCSFP163,MACCSFP164,MACCSFP165,MACCSFP166
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


In [42]:
#PubChem
fingerprint = 'PubChem'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_PubChem_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_PubChem_train.drop(['Name'],axis=1,inplace=True)
df_PubChem_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/PubChem_train.csv', index=False)
df_PubChem_train

,ID,SMILES,Permeability,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
fingerprint = 'PubChem'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_PubChem_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
df_PubChem_test.drop(['Name'],axis=1,inplace=True)
df_PubChem_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/PubChem_test.csv', index=False)
df_PubChem_test

,ID,SMILES,Permeability,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
#AtomPairs2DCount
fingerprint = 'AtomPairs2DCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_AtomPairs2DCount_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_AtomPairs2DCount_train.drop(['Name'],axis=1,inplace=True)
df_AtomPairs2DCount_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/AtomPairs2DCount_train.csv', index=False)
df_AtomPairs2DCount_train

,ID,SMILES,Permeability,APC2D1_C_C,APC2D1_C_N,APC2D1_C_O,APC2D1_C_S,APC2D1_C_P,APC2D1_C_F,APC2D1_C_Cl,...,APC2D10_I_I,APC2D10_I_B,APC2D10_I_Si,APC2D10_I_X,APC2D10_B_B,APC2D10_B_Si,APC2D10_B_X,APC2D10_Si_Si,APC2D10_Si_X,APC2D10_X_X
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,76.0,37.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,77.0,35.0,15.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,75.0,41.0,14.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,70.0,38.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,71.0,39.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,19.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,17.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,14.0,8.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,13.0,8.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
fingerprint = 'AtomPairs2DCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_AtomPairs2DCount_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
df_AtomPairs2DCount_test.drop(['Name'],axis=1,inplace=True)
df_AtomPairs2DCount_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/AtomPairs2DCount_test.csv', index=False)
df_AtomPairs2DCount_test

,ID,SMILES,Permeability,APC2D1_C_C,APC2D1_C_N,APC2D1_C_O,APC2D1_C_S,APC2D1_C_P,APC2D1_C_F,APC2D1_C_Cl,...,APC2D10_I_I,APC2D10_I_B,APC2D10_I_Si,APC2D10_I_X,APC2D10_B_B,APC2D10_B_Si,APC2D10_B_X,APC2D10_Si_Si,APC2D10_Si_X,APC2D10_X_X
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,75.0,38.0,16.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,69.0,39.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,73.0,37.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,75.0,39.0,14.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,66.0,39.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,21.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,21.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,19.0,9.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,20.0,9.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
#AtomPairs2D
fingerprint = 'AtomPairs2D'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_AtomPairs2D_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
df_AtomPairs2D_train.drop(['Name'],axis=1,inplace=True)
df_AtomPairs2D_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/AtomPairs2D_train.csv', index=False)
df_AtomPairs2D_train

,ID,SMILES,Permeability,AD2D1,AD2D2,AD2D3,AD2D4,AD2D5,AD2D6,AD2D7,...,AD2D771,AD2D772,AD2D773,AD2D774,AD2D775,AD2D776,AD2D777,AD2D778,AD2D779,AD2D780
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
fingerprint = 'AtomPairs2D'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
df_AtomPairs2D_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
df_AtomPairs2D_test.drop(['Name'],axis=1,inplace=True)
df_AtomPairs2D_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/AtomPairs2D_test.csv', index=False)
df_AtomPairs2D_test

,ID,SMILES,Permeability,AD2D1,AD2D2,AD2D3,AD2D4,AD2D5,AD2D6,AD2D7,...,AD2D771,AD2D772,AD2D773,AD2D774,AD2D775,AD2D776,AD2D777,AD2D778,AD2D779,AD2D780
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
#EState
fingerprint = 'EState'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
EState_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
EState_train.drop(['Name'],axis=1,inplace=True)
EState_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/EState_train.csv', index=False)
EState_train

,ID,SMILES,Permeability,EStateFP1,EStateFP2,EStateFP3,EStateFP4,EStateFP5,EStateFP6,EStateFP7,...,EStateFP70,EStateFP71,EStateFP72,EStateFP73,EStateFP74,EStateFP75,EStateFP76,EStateFP77,EStateFP78,EStateFP79
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [49]:
fingerprint = 'EState'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
EState_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
EState_test.drop(['Name'],axis=1,inplace=True)
EState_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/EState_test.csv', index=False)
EState_test

,ID,SMILES,Permeability,EStateFP1,EStateFP2,EStateFP3,EStateFP4,EStateFP5,EStateFP6,EStateFP7,...,EStateFP70,EStateFP71,EStateFP72,EStateFP73,EStateFP74,EStateFP75,EStateFP76,EStateFP77,EStateFP78,EStateFP79
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [50]:
#Extended
fingerprint = 'Extended'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Extended_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
Extended_train.drop(['Name'],axis=1,inplace=True)
Extended_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/Extended_train.csv', index=False)
Extended_train

,ID,SMILES,Permeability,ExtFP1,ExtFP2,ExtFP3,ExtFP4,ExtFP5,ExtFP6,ExtFP7,...,ExtFP1015,ExtFP1016,ExtFP1017,ExtFP1018,ExtFP1019,ExtFP1020,ExtFP1021,ExtFP1022,ExtFP1023,ExtFP1024
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,0,1,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
fingerprint = 'Extended'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Extended_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
Extended_test.drop(['Name'],axis=1,inplace=True)
Extended_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/Extended_test.csv', index=False)
Extended_test

,ID,SMILES,Permeability,ExtFP1,ExtFP2,ExtFP3,ExtFP4,ExtFP5,ExtFP6,ExtFP7,...,ExtFP1015,ExtFP1016,ExtFP1017,ExtFP1018,ExtFP1019,ExtFP1020,ExtFP1021,ExtFP1022,ExtFP1023,ExtFP1024
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,0,1,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [52]:
#Fingerprinter
fingerprint = 'Fingerprinter'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Fingerprinter_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
Fingerprinter_train.drop(['Name'],axis=1,inplace=True)
Fingerprinter_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/Fingerprinter_train.csv', index=False)
Fingerprinter_train

,ID,SMILES,Permeability,FP1,FP2,FP3,FP4,FP5,FP6,FP7,...,FP1015,FP1016,FP1017,FP1018,FP1019,FP1020,FP1021,FP1022,FP1023,FP1024
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [53]:
fingerprint = 'Fingerprinter'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Fingerprinter_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
Fingerprinter_test.drop(['Name'],axis=1,inplace=True)
Fingerprinter_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/Fingerprinter_test.csv', index=False)
Fingerprinter_test

,ID,SMILES,Permeability,FP1,FP2,FP3,FP4,FP5,FP6,FP7,...,FP1015,FP1016,FP1017,FP1018,FP1019,FP1020,FP1021,FP1022,FP1023,FP1024
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
#GraphOnly
fingerprint = 'Graphonly'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Graphonly_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
Graphonly_train.drop(['Name'],axis=1,inplace=True)
Graphonly_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/Graphonly_train.csv', index=False)
Graphonly_train

,ID,SMILES,Permeability,GraphFP1,GraphFP2,GraphFP3,GraphFP4,GraphFP5,GraphFP6,GraphFP7,...,GraphFP1015,GraphFP1016,GraphFP1017,GraphFP1018,GraphFP1019,GraphFP1020,GraphFP1021,GraphFP1022,GraphFP1023,GraphFP1024
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
fingerprint = 'Graphonly'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
Graphonly_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
Graphonly_test.drop(['Name'],axis=1,inplace=True)
Graphonly_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/Graphonly_test.csv', index=False)
Graphonly_test

,ID,SMILES,Permeability,GraphFP1,GraphFP2,GraphFP3,GraphFP4,GraphFP5,GraphFP6,GraphFP7,...,GraphFP1015,GraphFP1016,GraphFP1017,GraphFP1018,GraphFP1019,GraphFP1020,GraphFP1021,GraphFP1022,GraphFP1023,GraphFP1024
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
#KlekotaRothCount
fingerprint = 'KlekotaRothCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
KlekotaRothCount_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
KlekotaRothCount_train.drop(['Name'],axis=1,inplace=True)
KlekotaRothCount_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/KlekotaRothCount_train.csv', index=False)
KlekotaRothCount_train

,ID,SMILES,Permeability,KRFPC1,KRFPC2,KRFPC3,KRFPC4,KRFPC5,KRFPC6,KRFPC7,...,KRFPC4851,KRFPC4852,KRFPC4853,KRFPC4854,KRFPC4855,KRFPC4856,KRFPC4857,KRFPC4858,KRFPC4859,KRFPC4860
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,21.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,19.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,21.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,23.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
fingerprint = 'KlekotaRothCount'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
KlekotaRothCount_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
KlekotaRothCount_test.drop(['Name'],axis=1,inplace=True)
KlekotaRothCount_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/KlekotaRothCount_test.csv', index=False)
KlekotaRothCount_test

,ID,SMILES,Permeability,KRFPC1,KRFPC2,KRFPC3,KRFPC4,KRFPC5,KRFPC6,KRFPC7,...,KRFPC4851,KRFPC4852,KRFPC4853,KRFPC4854,KRFPC4855,KRFPC4856,KRFPC4857,KRFPC4858,KRFPC4859,KRFPC4860
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,22.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,21.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,22.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,17.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,21.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
#KlekotaRoth
fingerprint = 'KlekotaRoth'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_train.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/train.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
KlekotaRoth_train = pd.concat([df_train[['ID','SMILES','Permeability']], descriptors], axis=1)
KlekotaRoth_train.drop(['Name'],axis=1,inplace=True)
KlekotaRoth_train.to_csv('/home/users/akshay/PCPpred/Fingerprints/Train/KlekotaRoth_train.csv', index=False)
KlekotaRoth_train

,ID,SMILES,Permeability,KRFP1,KRFP2,KRFP3,KRFP4,KRFP5,KRFP6,KRFP7,...,KRFP4851,KRFP4852,KRFP4853,KRFP4854,KRFP4855,KRFP4856,KRFP4857,KRFP4858,KRFP4859,KRFP4860
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
fingerprint = 'KlekotaRoth'
fingerprint_output_file = ''.join(['/home/users/akshay/PCPpred/Fingerprints/',fingerprint,'_test.csv']) 
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='/home/users/akshay/PCPpred/data/test.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

descriptors = pd.read_csv(fingerprint_output_file)
KlekotaRoth_test = pd.concat([df_test[['ID','SMILES','Permeability']], descriptors], axis=1)
KlekotaRoth_test.drop(['Name'],axis=1,inplace=True)
KlekotaRoth_test.to_csv('/home/users/akshay/PCPpred/Fingerprints/Test/KlekotaRoth_test.csv', index=False)
KlekotaRoth_test

,ID,SMILES,Permeability,KRFP1,KRFP2,KRFP3,KRFP4,KRFP5,KRFP6,KRFP7,...,KRFP4851,KRFP4852,KRFP4853,KRFP4854,KRFP4855,KRFP4856,KRFP4857,KRFP4858,KRFP4859,KRFP4860
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_train_morgan = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/morgan_fp_train.csv')
df_train_morganCount = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/count_morgan_fp_train.csv')
df_train_AP2d = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/AtomPairs2D_train.csv')
df_train_AP2dCount = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/AtomPairs2DCount_train.csv')
df_train_EState = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/EState_train.csv')
df_train_Extended = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/Extended_train.csv')
df_train_fp= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/Fingerprinter_train.csv')
df_train_graph= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/Graphonly_train.csv')
df_train_kr= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/KlekotaRoth_train.csv')
df_train_krcount= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/KlekotaRothCount_train.csv')
df_train_maccs= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/MACCS_train.csv')
df_train_pubchem= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/PubChem_train.csv')
df_train_Substr= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/Substructure_train.csv')
df_train_Substrcount= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/SubstructureCount_train.csv')

df = df_train_morgan.merge(df_train_morganCount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_AP2d, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_AP2dCount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_EState, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_Extended, on=['ID', 'SMILES', 'Permeability'], how='inner')
df = df.merge(df_train_fp, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_graph, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_kr, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_krcount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_maccs, on=['ID', 'SMILES', 'Permeability'], how='inner')
df = df.merge(df_train_pubchem, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_Substr, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_train_Substrcount, on=['ID', 'SMILES', 'Permeability'], how='inner')
df

,ID,SMILES,Permeability,Morgan_fp_0,Morgan_fp_1,Morgan_fp_2,Morgan_fp_3,Morgan_fp_4,Morgan_fp_5,Morgan_fp_6,...,SubFPC298,SubFPC299,SubFPC300,SubFPC301,SubFPC302,SubFPC303,SubFPC304,SubFPC305,SubFPC306,SubFPC307
0,915,CC[C@H](C)[C@H](NC(=O)[C@@H]1CC(=O)N[C@@H](Cc2...,-7.0,0,1,1,0,0,1,0,...,0.0,0.0,90.0,90.0,30.0,0.0,0.0,0.0,0.0,70.0
1,888,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1cccc(Cl)c1)N(C)...,-7.0,0,1,1,0,0,1,0,...,0.0,0.0,82.0,82.0,36.0,0.0,0.0,0.0,0.0,71.0
2,593,C/N=C(\NC)NCCC[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C...,-7.0,0,1,1,0,0,1,0,...,0.0,0.0,84.0,84.0,32.0,0.0,0.0,0.0,0.0,72.0
3,916,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@@H]1CC(=...,-7.0,0,1,1,0,0,1,0,...,0.0,0.0,90.0,90.0,28.0,0.0,0.0,0.0,0.0,64.0
4,900,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccccc1)N(C)C(=O...,-7.0,0,1,1,0,0,1,0,...,0.0,0.0,90.0,90.0,37.0,0.0,0.0,0.0,0.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.7,0,1,0,0,0,0,0,...,0.0,0.0,21.0,21.0,4.0,0.0,0.0,0.0,0.0,17.0
5564,2467,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-5.6,0,1,0,0,0,0,0,...,0.0,0.0,19.0,19.0,4.0,0.0,0.0,0.0,0.0,17.0
5565,2512,CC(C)C[C@H]1NC(=O)[C@@H](C)NCCCCCCNC(=O)[C@H](...,-5.5,0,1,1,0,0,0,0,...,0.0,0.0,23.0,23.0,3.0,0.0,0.0,0.0,0.0,11.0
5566,2511,CC(C)[C@@H]1NC(=O)[C@@H](CO)NC(=O)[C@@H](C)NCC...,-6.3,0,1,1,0,0,0,0,...,0.0,0.0,22.0,22.0,2.0,0.0,0.0,0.0,0.0,11.0


In [6]:
df.to_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/All_fingerprints_train.csv',index=False)

In [8]:
df_test_morgan = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/morgan_fp_test.csv')
df_test_morganCount = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/count_morgan_fp_test.csv')
df_test_AP2d = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/AtomPairs2D_test.csv')
df_test_AP2dCount = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/AtomPairs2DCount_test.csv')
df_test_EState = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/EState_test.csv')
df_test_Extended = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/Extended_test.csv')
df_test_fp= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/Fingerprinter_test.csv')
df_test_graph= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/Graphonly_test.csv')
df_test_kr= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/KlekotaRoth_test.csv')
df_test_krcount= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/KlekotaRothCount_test.csv')
df_test_maccs= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/MACCS_test.csv')
df_test_pubchem= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/PubChem_test.csv')
df_test_Substr= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/Substructure_test.csv')
df_test_Substrcount= pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/SubstructureCount_test.csv')

df = df_test_morgan.merge(df_test_morganCount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_AP2d, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_AP2dCount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_EState, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_Extended, on=['ID', 'SMILES', 'Permeability'], how='inner')
df = df.merge(df_test_fp, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_graph, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_kr, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_krcount, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_maccs, on=['ID', 'SMILES', 'Permeability'], how='inner')
df = df.merge(df_test_pubchem, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_Substr, on=['ID', 'SMILES', 'Permeability'], how='inner').merge(df_test_Substrcount, on=['ID', 'SMILES', 'Permeability'], how='inner')
df

,ID,SMILES,Permeability,Morgan_fp_0,Morgan_fp_1,Morgan_fp_2,Morgan_fp_3,Morgan_fp_4,Morgan_fp_5,Morgan_fp_6,...,SubFPC298,SubFPC299,SubFPC300,SubFPC301,SubFPC302,SubFPC303,SubFPC304,SubFPC305,SubFPC306,SubFPC307
0,908,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-7.00,0,1,1,0,0,1,0,...,0.0,0.0,88.0,88.0,34.0,0.0,0.0,0.0,0.0,70.0
1,923,CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)[C@H](...,-7.00,0,1,1,0,0,1,0,...,0.0,0.0,89.0,89.0,25.0,0.0,0.0,0.0,0.0,64.0
2,897,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.00,0,1,1,0,0,1,0,...,0.0,0.0,86.0,86.0,35.0,0.0,0.0,0.0,0.0,70.0
3,587,CC(C)C[C@@H]1NC(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@...,-6.74,0,1,1,0,0,1,0,...,0.0,0.0,80.0,80.0,28.0,0.0,0.0,0.0,0.0,73.0
4,921,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.54,0,1,1,0,0,1,0,...,0.0,0.0,86.0,86.0,20.0,0.0,0.0,0.0,0.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,0,1,1,0,0,0,0,...,0.0,0.0,23.0,23.0,4.0,0.0,0.0,0.0,0.0,17.0
1388,2485,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@@H](Cc2cccc...,-4.80,0,1,1,0,0,0,0,...,0.0,0.0,23.0,23.0,4.0,0.0,0.0,0.0,0.0,17.0
1389,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,0,1,0,0,0,1,0,...,0.0,0.0,21.0,21.0,4.0,0.0,0.0,0.0,0.0,17.0
1390,2513,C[C@H]1NCCCCCCNC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[...,-7.80,0,0,1,0,0,1,0,...,0.0,0.0,23.0,23.0,2.0,0.0,0.0,0.0,0.0,16.0


In [9]:
df.to_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/All_fingerprints_test.csv',index=False)